In [ ]:
import torch

In [3]:
def reluToInplaceFalse(model):
  for name, child in model.named_children():
    if isinstance(child, nn.ReLU):
      setattr(child, 'inplace', False)
    else:
      reluToInplaceFalse(child)

from torchvision.transforms.transforms import RandomRotation

class Classifier(torch.nn.Module):

  def __init__(self, backbone='resnet', multi_backbone = True, device ="cuda:0",dropout_rate = 0.2, do_augmentation = False):
    super().__init__()
    self.multi_backbone = multi_backbone

    if backbone == "vgg19":
      backbone = torchvision.models.vgg19(pretrained=True)
      self.out_channels = 25088
      
    elif backbone == "resnet18":
      backbone = torchvision.models.resnet18(pretrained=True)
      self.out_channels = 512

    elif backbone == "resnet50":
      backbone = torchvision.models.resnet50(pretrained=True)
      self.out_channels = 2048

    elif backbone == "Efficientnet b1":
      backbone = torchvision.models.efficientnet_b1(pretrained=True)
      self.out_channels = 1280

    elif backbone == "Efficientnet b3":
      backbone = torchvision.models.efficientnet_b3(pretrained=True)
      self.out_channels = 1536

    elif backbone == "Efficientnet b5":
      backbone = torchvision.models.efficientnet_b5(pretrained=True)
      self.out_channels = 2048

    elif backbone == "Efficientnet b7":
      backbone = torchvision.models.efficientnet_b7(pretrained=True)
      self.out_channels = 2560
      
    # Disabling inplace ReLu becasuse GradCam doesn't work it enabled
    reluToInplaceFalse(backbone)
     
    modules = list(backbone.children())[:-1]
    self.do_augmentation = do_augmentation

    if self.do_augmentation:
      self.augmentation = nn.Sequential(transforms.RandomHorizontalFlip(),
                                        transforms.RandomVerticalFlip(),
                                        transforms.RandomPerspective(0.2),
                                        RandomRotation(20),
                                        transforms.RandomAutocontrast())

    if self.multi_backbone:
      self.backbone1 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone2 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone3 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone4 = nn.Sequential(*copy.deepcopy(modules)).to(device)
    else:
      self.backbone =  nn.Sequential(*modules).to(device)


     

    self.fc1 = nn.Sequential(nn.Dropout(dropout_rate),
                              nn.Linear(self.out_channels, 128),
                              nn.ReLU(),
                              nn.Dropout(dropout_rate)) #TODO: Experiment with BN and Dropout

    # 512 features in, 3 features out
    self.fc = nn.Sequential(nn.Linear(512, 3))                  #TODO: L2 Regularization
     
  def forward(self, x, is_training = True):
    if self.do_augmentation and is_training:
      imgs = [self.augmentation(x[:,i]) for i in range(4)] #list of 4 images
    else:
      imgs = [x[:,i] for i in range(4)] #list of 4 images
      #imgs = [x[i] for i in range(4)]

    if self.multi_backbone:
        # feed each image into a backbone
      encodings = [self.fc1(torch.flatten(self.backbone1(imgs[0]),1)),
                   self.fc1(torch.flatten(self.backbone2(imgs[1]),1)),
                   self.fc1(torch.flatten(self.backbone3(imgs[2]),1)),
                   self.fc1(torch.flatten(self.backbone4(imgs[3]),1))]
    else:
      encodings = [self.fc1(self.backbone(img).squeeze()) for img in imgs]

    # modify to return an array with the largest encoding
    
    #raw_result = self.fc(torch.cat(encodings,1))
    #fixed_result = torch.clone(raw_result)
    #for idx, row in enumerate(raw_result):
    #    ones = torch.ones(3, dtype=torch.long)
    #    max_val = row.max()
    #    fixed_result[idx] = torch.where(row < max_val, 0, ones)
                  
    #return fixed_result.type(torch.int64)
    return self.fc(torch.cat(encodings,1))

In [4]:
model = torch.load('model_b_mar_7_00_05.pt', map_location="cpu")
model.eval()
model.zero_grad()

In [5]:
import numpy as np
x = np.load("x_train_b.npy")
y = np.load("y_train_b.npy")

from sklearn.model_selection import train_test_split

tensor_x = torch.Tensor(x) 
tensor_y = torch.Tensor(y).long()


tensor_x = torch.swapaxes(tensor_x,2,4)
tensor_x = torch.swapaxes(tensor_x,3,4)

x_train,x_test,y_train,y_test = train_test_split(tensor_x,tensor_y,test_size=0.1)

print("x train shape: ", x_train.shape,"x test shape: ",x_test.shape)

import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
train_ds = TensorDataset(x_train,nn.functional.one_hot(y_train,3)) 
test_ds = TensorDataset(x_test,nn.functional.one_hot(y_test,3)) 

del x_train, x_test, y_train, y_test

train_dl = DataLoader(train_ds,4,shuffle = True)
test_dl = DataLoader(test_ds,4,drop_last = True)
exp_dl = DataLoader(test_ds,10,drop_last = True)

del x,y

x train shape:  torch.Size([256, 4, 3, 200, 1024]) x test shape:  torch.Size([29, 4, 3, 200, 1024])


In [6]:
instances, labels = iter(exp_dl).next()

In [7]:
new_labels = []
for t in labels.squeeze():
  if torch.equal(t, torch.tensor([1,0,0])):
    new_labels.append(0)
  elif torch.equal(t, torch.tensor([0,1,0])):
    new_labels.append(1)
  elif torch.equal(t, torch.tensor([0,0,1])):
    new_labels.append(2)

In [8]:
new_labels

[2, 1, 2, 0, 1, 2, 0, 1, 0, 1]

In [9]:
# attribution for model
from captum.attr import ShapleyValueSampling
svs = ShapleyValueSampling(model)

In [ ]:
#attributions = svs.attribute(images[:50], target=labels[:50], feature_mask=feature_mask_14x1#4, n_samples = 10, show_progress = True)
attributions = svs.attribute(instances, target=new_labels, n_samples = 5, show_progress = True)

In [10]:
# create feature masks, check image dimensions
instances, labels = iter(exp_dl).next()

In [11]:
instances.shape

torch.Size([10, 4, 3, 200, 1024])

In [12]:
instance = instances[0]
instance.shape

torch.Size([4, 3, 200, 1024])

In [13]:
labels.shape

torch.Size([10, 1, 3])

In [14]:
# 20x16 pixel basis blocks
base_matrix = np.array([0] * 320).reshape(20,16)
base_row_list = []
for i in range(64):
    base_row_list.append(base_matrix + i)

base_row = np.hstack((tuple(base_row_list)))
all_rows = []
for i in range(10):
    all_rows.append(base_row + (64 * i))

one_channel_mat = np.vstack(tuple(all_rows))
# now we need to create three instances of one_channel_mat

In [15]:
one_channel_mat.shape

(200, 1024)

In [16]:
a = np.zeros((3, 200, 1024))

In [17]:
a[0] = one_channel_mat 
a[1] = one_channel_mat
a[2] = one_channel_mat

In [18]:
b = np.zeros((4, 3, 200, 1024))
# should be distinct values
b[0] = a
b[1] = a
b[2] = a
b[3] = a

In [19]:
b.shape

(4, 3, 200, 1024)

In [20]:
feature_mask_20x16 = torch.tensor(b)

In [ ]:
attributions = svs.attribute(instances, target=new_labels, feature_mask = feature_mask_20x16, n_samples = 5, show_progress = True, perturbations_per_eval = 10)